In [2]:
import sys
import time
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import time
from pandas import Series, DataFrame
GDriveJSON = 'PythonUpload.json'
GSpreadSheet = 'stock'
try:
    scope =['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    key = SAC.from_json_keyfile_name(GDriveJSON, scope)
    gc = gspread.authorize(key)
    worksheet = gc.open(GSpreadSheet).worksheet("3545")
except Exception as ex:
    print('無法連線Google試算表', ex)
    sys.exit(1)
else:
    data = worksheet.get_all_values()

In [3]:
data = DataFrame(data)
data.columns = data.ix[0]

C:\Users\Chueh\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [4]:
data.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,成交筆數
0,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,成交筆數
1,107/05/03,2277731,59023878,26.2,26.4,25.7,25.8,1236
2,107/05/02,1979457,51946823,26.3,26.45,26.1,26.1,1267
3,107/04/30,4512429,117309604,26.5,26.5,25.6,25.95,2506
4,107/04/27,2804362,76649003,27.75,28.25,26.6,27,1362


In [5]:
data = data[1:]

In [6]:
stock_data_df = data.copy()
def replace_year(x):
    if len(x)==9:
        return x.replace(x[0:3],str(int(x[0:3])+1911))
    else:
        return x.replace(x[0:2],str(int(x[0:2])+1911))
stock_data_df['日期'] = stock_data_df['日期'].apply(replace_year)

In [7]:
stock_data_df.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,成交筆數
1,2018/05/03,2277731,59023878,26.2,26.4,25.7,25.8,1236
2,2018/05/02,1979457,51946823,26.3,26.45,26.1,26.1,1267
3,2018/04/30,4512429,117309604,26.5,26.5,25.6,25.95,2506
4,2018/04/27,2804362,76649003,27.75,28.25,26.6,27,1362
5,2018/04/26,3198776,90291614,28.25,29.2,27.3,27.4,1464


In [8]:
stock_data_df.index=stock_data_df['日期']
stock_data_df.drop(['日期','成交金額','成交筆數'],axis=1,inplace=True)


In [9]:
stock_data_df.columns=['volume','open','high','low','close']
stock_data_df.head()

,volume,open,high,low,close
日期,,,,,
2018/05/03,2277731,26.2,26.4,25.7,25.8
2018/05/02,1979457,26.3,26.45,26.1,26.1
2018/04/30,4512429,26.5,26.5,25.6,25.95
2018/04/27,2804362,27.75,28.25,26.6,27
2018/04/26,3198776,28.25,29.2,27.3,27.4


In [10]:
#資料正規化
from sklearn import preprocessing

def normalize(df):
    newdf= df.copy()
    min_max_scaler = preprocessing.MinMaxScaler()
    
    newdf['open'] = min_max_scaler.fit_transform(df.open.values.reshape(-1,1))
    newdf['low'] = min_max_scaler.fit_transform(df.low.values.reshape(-1,1))
    newdf['high'] = min_max_scaler.fit_transform(df.high.values.reshape(-1,1))
    newdf['volume'] = min_max_scaler.fit_transform(df.volume.values.reshape(-1,1))
    newdf['close'] = min_max_scaler.fit_transform(df.close.values.reshape(-1,1))
    
    return newdf
stock_data_norm= normalize(stock_data_df)

C:\Users\Chueh\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
stock_data_norm.head()

,volume,open,high,low,close
日期,,,,,
2018/05/03,0.085399,0.133005,0.096346,0.131716,0.101255
2018/05/02,0.073142,0.134647,0.097176,0.138648,0.106276
2018/04/30,0.177228,0.137931,0.098007,0.129983,0.103766
2018/04/27,0.107040,0.158456,0.127076,0.147314,0.121339
2018/04/26,0.123247,0.166667,0.142857,0.159445,0.128033


In [12]:
stock_data_norm.index = pd.to_datetime(stock_data_norm.index)

In [13]:
def split_test_data(df,time_frame,train_ratio=0.9):
    number_features = len(df.columns)
    
    datavalue = df.as_matrix()
    
    result=[]
    
    for index in range(len(datavalue) - (time_frame+1) ):
        result.append(datavalue[index: index + (time_frame+1)])
    
    result = np.array(result)
    number_train = round(train_ratio * result.shape[0])
    
    x_train = result[:int(number_train),:-1]
    y_train = result[:int(number_train),-1][:,-1]
    
    x_test = result[int(number_train):,:-1]
    y_test = result[int(number_train):,-1][:,-1]
    
    x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],number_features))
    x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],number_features))
    
    return [x_train, y_train, x_test, y_test]

In [14]:
X_train, y_train, X_test, y_test = split_test_data(stock_data_norm, 20)

In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import keras

C:\Users\Chueh\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
def build_model(input_length, input_dim):
    d = 0.3
    model = Sequential()
    model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=True))
    model.add(Dropout(d))
    model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=False))
    model.add(Dropout(d))
    model.add(Dense(16,kernel_initializer="uniform",activation='relu'))
    model.add(Dense(1,kernel_initializer="uniform",activation='linear'))
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    return model
model = build_model( 20, 5 )

In [ ]:

# 一個batch有128個instance，總共跑50個迭代
model.fit( X_train, y_train, batch_size=128, epochs=50, validation_split=0.1, verbose=1)

Train on 1843 samples, validate on 205 samples
Epoch 1/50
1843/1843 [==============================] - 12s 7ms/step - loss: 0.0710 - acc: 5.4259e-04 - val_loss: 0.1198 - val_acc: 0.0000e+00
Epoch 2/50
1843/1843 [==============================] - 12s 6ms/step - loss: 0.0207 - acc: 5.4259e-04 - val_loss: 0.0254 - val_acc: 0.0049
Epoch 3/50
1843/1843 [==============================] - 13s 7ms/step - loss: 0.0050 - acc: 5.4259e-04 - val_loss: 0.0071 - val_acc: 0.0049
Epoch 4/50
1843/1843 [==============================] - 10s 6ms/step - loss: 0.0032 - acc: 5.4259e-04 - val_loss: 0.0070 - val_acc: 0.0049
Epoch 5/50
1843/1843 [==============================] - 10s 6ms/step - loss: 0.0028 - acc: 5.4259e-04 - val_loss: 0.0071 - val_acc: 0.0049
Epoch 6/50
1843/1843 [==============================] - 10s 6ms/step - loss: 0.0028 - acc: 5.4259e-04 - val_loss: 0.0057 - val_acc: 0.0049
Epoch 7/50
1843/1843 [==============================] - 10s 6ms/step - loss: 0.0024 - acc: 5.4259e-04 - val_loss: 0

In [17]:
def denormalize(df, norm_value):
    original_value = df['close'].values.reshape(-1,1)
    norm_value = norm_value.reshape(-1,1)
    
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit_transform(original_value)
    denorm_value = min_max_scaler.inverse_transform(norm_value)
    
    return denorm_value
# 用訓練好的 LSTM 模型對測試資料集進行預測
pred = model.predict(X_test)
# 將預測值與正確答案還原回原來的區間值
denorm_pred = denormalize(stock_data_df, pred)
denorm_ytest = denormalize(stock_data_df, y_test)


C:\Users\Chueh\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  
plt.plot(denorm_pred,color='red', label='Prediction')
plt.plot(denorm_ytest,color='blue', label='Answer')
plt.legend(loc='best')
plt.show()

NameError: name 'denorm_pred' is not defined

In [50]:
a = DataFrame(denorm_pred)
b = DataFrame(denorm_ytest)
#np.corrcoef(denorm_pred[0],denorm_ytest[0])
pd.concat([a,b]).corr()

,0
0,1.0


In [46]:
b

,0
0,63.10
1,59.00
2,56.30
3,59.70
4,59.70
5,59.40
6,62.00
7,61.00
8,57.10
9,56.10
